##### Extracting and cleaning the data

<p> Installing and importing necessary libraries. </p>

In [1]:
!pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup  # To work with a HTML page
import pandas as pd
import numpy as np

Extracting content from a given url and stroring it using BeutifulfulSoup

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
# print(r.content) 
soup = BeautifulSoup(r.content, 'html5lib') 
# print(soup.prettify()) 

Dictionary to store the content of a page so that it can be directly converted to Dataframe for an easy manipulation.

In [3]:
from collections import defaultdict
dic = defaultdict(list)

Extracting data from a table tag with help of BeautifulSoup and storing it in a dictionary

In [4]:
table = soup.table
rows = table.find_all('tr')
for r in rows:
    cols = r.find_all('td')
#     row = [ dict[]=i.text[:-1] for i in cols]
    for i in range(len(cols)):
        if i == 0:
            dic['Postal Code'].append(cols[i].text[:-1])
        if i == 1:
            dic['Borough'].append(cols[i].text[:-1])
        if i == 2:
            dic['Neighborhood'].append(cols[i].text[:-1])
# dic          

Converting Dictionary to Dataframe.

In [5]:
data = pd.DataFrame.from_dict(dic)
data

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Extracting Boroughs which are 'Not assigned'

In [6]:
to_remove = data[data['Borough'] == 'Not assigned'].index

Dropping the Boroughs which are 'Not assigned'

In [7]:
data.drop(to_remove, inplace = True)
data

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Resetting index.

In [8]:
data.reset_index(inplace = True)
data.head()

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
Dropping the unnecessary index column.

In [9]:
data.drop('index', axis = 1)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Shape of the final DataFrame.

In [10]:
data.shape

(103, 4)

Installing GeoCoder to get the latitude and longitude for Boroughs in 'data' Dataframe 

In [12]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 5.3MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 829kB 8.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


The below cell didn't worked as API was taking too much time.

In [13]:
# latitude = []
# longitude = []
# for pcode in data['Postal Code']:
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#       response = geocoder.google('{}, Toronto, Ontario'.format(pcode))
#       lat_lng_coords = response.latlng

#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

So, I read latitudes and logitudes from a csv file

In [14]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sorting data and removing any trailing white spaces in columns

In [15]:
data.columns = data.columns.str.strip()
data.sort_values('Postal Code', ascending = True, axis = 0, inplace = True)
data.head()

,index,Postal Code,Borough,Neighborhood
6,9,M1B,Scarborough,"Malvern, Rouge"
12,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,36,M1G,Scarborough,Woburn
26,45,M1H,Scarborough,Cedarbrae


Sorting lat_long inorder to merge it with data so each Borough get its latitude and longitude correctly

In [16]:
lat_long.columns = lat_long.columns.str.strip()
lat_long.sort_values('Postal Code', ascending = True, axis = 0, inplace = True)
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adding Latitude and Longitude columns to 'data' dataframe.

In [17]:
data['Latitude'] = lat_long['Latitude']
data['Longitude'] = lat_long['Longitude']
data.head()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
6,9,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
12,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
18,27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
22,36,M1G,Scarborough,Woburn,43.770120,-79.408493
26,45,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


In [18]:
data.reset_index(inplace = True)
data.head()

,level_0,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,6,9,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,12,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,18,27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,22,36,M1G,Scarborough,Woburn,43.770120,-79.408493
4,26,45,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


Removing extra columns. It may not look exactly like given in assigment as it is sorted.

In [22]:
data.drop(['level_0', 'index'], axis = 1, inplace = True)
data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.709060,-79.363452
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.728020,-79.388790
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.667967,-79.367675
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.650571,-79.384568
